In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100, 'display.max_rows', 150)
import numpy as np
import matplotlib.pyplot as plt
import config

#Load google.cloud.bigquery
%load_ext google.cloud.bigquery

#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [2]:
%%bigquery --use_rest_api ZRI_MF
SELECT *
FROM `high-empire-220313.ZRI.Multi_Family`

In [3]:
def month_year(x):
    month = int(x[-2:])
    year = int(x[1:5])
    if year<2014:
        return(True)
    if (year == 2014) and month<3:
        return(True)
    return(False)

In [4]:
time_columns = [x for x in ZRI_MF.columns if ('20' in x)]
time_drop = [x for x in time_columns if month_year(x)]

In [5]:
ZRI_MF = ZRI_MF.drop(time_drop, axis = 1)

In [6]:
ZRI_MF['Missing_Months'] = ZRI_MF.isna().apply(sum, axis = 1)

In [7]:
def consecutive_missing(row):
    return(max(row.isnull().astype(int).groupby(row.notnull().astype(int).cumsum()).sum()))

In [8]:
ZRI_MF.apply(consecutive_missing,axis = 1).value_counts().sort_index(ascending= False)

70      8
69     25
68     24
67     12
66     20
65     20
64     17
63     27
62     22
61     15
60     15
59      6
57      1
53      1
52     11
51     52
49      1
46      8
45      1
44      1
42     10
40      1
35      4
34     24
31      1
30      1
28      1
25      1
24      1
23     11
22     35
21      1
20      1
19      1
18      4
17      1
16      1
15      5
14      8
13     29
12      9
11     12
10     37
9      14
8      16
7      19
6      22
5      23
4      48
3      96
2     106
1     393
0     638
dtype: int64

In [15]:
ZRI_filtered = ZRI_MF[ZRI_MF.apply(consecutive_missing,axis = 1)<5]

In [17]:
ZRI_filtered.loc[:,[x for x in ZRI_MF.columns if ('20' in x)]] = ZRI_filtered[[x for x in ZRI_MF.columns if ('20' in x)]].interpolate(method = 'linear',
                                                                     axis = 0)

c:\users\msend\projects\nyc_dsa\7parkmgm\venv\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [18]:
ZRI_filtered = ZRI_filtered.drop('Metro', axis = 1)

ZRI_filtered.to_pickle('./pickles/ZRI_filtered.p')
import pickle
file = open('./pickles/ZRI_filtered.p','rb')
filtered2 = pickle.load(file)